# 04 — Generate Streamlit Options (Crime DB)

This notebook reads **data/crime.db** and writes dropdown options for the Streamlit UI to `streamlit/options.json`.

Run this after you build the database and before deploying.

In [1]:
import json, sqlite3
from pathlib import Path
import pandas as pd

DB_PATH = "data/crime.db"
OUT_PATH = Path("streamlit/options.json")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

SQL = """
SELECT
  i.zip_code,
  i.council_district,
  dow.name AS day_of_week,
  pd.name  AS police_district,
  nb.name  AS neighborhood
FROM incidents i
JOIN labels l ON l.incident_id = i.incident_id
LEFT JOIN day_of_week_dim dow ON dow.day_of_week_id = i.day_of_week_id
LEFT JOIN police_district_dim pd ON pd.police_district_id = i.police_district_id
LEFT JOIN neighborhood_dim nb ON nb.neighborhood_id = i.neighborhood_id;
"""

con = sqlite3.connect(DB_PATH)
df = pd.read_sql_query(SQL, con)
con.close()

df.head()


,zip_code,council_district,day_of_week,police_district,neighborhood
0,14202,FILLMORE,Friday,District B,Central
1,14214,MASTEN,Tuesday,District E,Fillmore-Leroy
2,14216,NORTH,Tuesday,District D,Parkside
3,14211,LOVEJOY,Saturday,District C,Genesee-Moselle
4,14207,NORTH,Monday,District D,Riverside


In [2]:
def clean_unique(series):
    s = series.dropna().astype(str).str.strip()
    s = s[s != ""]
    return sorted(s.unique().tolist())

options = {
    "zip_code": clean_unique(df["zip_code"]),
    "council_district": clean_unique(df["council_district"]),
    "day_of_week": clean_unique(df["day_of_week"]),
    "police_district": clean_unique(df["police_district"]),
    "neighborhood": clean_unique(df["neighborhood"]),
}

OUT_PATH.write_text(json.dumps(options, indent=2), encoding="utf-8")
print("✅ Wrote:", OUT_PATH.resolve())
{k: len(v) for k, v in options.items()}


✅ Wrote: /Users/kavyansh/IdeaProjects/ml_project/buffalo_crime/streamlit/options.json


{'zip_code': 21,
 'council_district': 10,
 'day_of_week': 7,
 'police_district': 6,
 'neighborhood': 36}